In [1]:
import pandas as pd
import datetime
from re import sub
from decimal import Decimal

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.downloader.download('vader_lexicon')

from langdetect import detect

from sklearn import preprocessing

import warnings
warnings.filterwarnings("ignore")

import boto3
from boto3 import session
from sagemaker import get_execution_role 

import seaborn as sns
import matplotlib as plt

# Change pandas viewing options
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/danielacollaguazo/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# import sys
# !{sys.executable} -m pip install sklearn

In [6]:
df_calendar = pd.read_csv('../../data/new-york-city-airbnb-open-data/calendar.csv')

In [9]:
df_calendar.available.value_counts(dropna=False)

f    12431816
t     6108724
Name: available, dtype: int64

## Count the number of unique listings in calendar and listings DF

In [3]:
df_calendar['listing_id'].nunique()

50796

In [15]:
# list(df_listings.columns)
df_listings = df_listings.rename(columns={"id": "listing_id"})

## Joining calendar with listings. I want to know data from what years are available

In [17]:
df_listings_date = pd.merge(df_listings, df_calendar, how='inner', on=['listing_id', 'listing_id'])

In [18]:
df_calendar['date'] = pd.to_datetime(df_calendar['date'])
df_calendar ['year'] = pd.DatetimeIndex(df_calendar['date']).year

## Years of listings available

In [19]:
pd.Categorical(df_calendar['year'])

[2020, 2020, 2020, 2020, 2020, ..., 2021, 2021, 2021, 2021, 2021]
Length: 18540540
Categories (2, int64): [2020, 2021]

## Creating season label

In [20]:
df_calendar['season'] = (pd.DatetimeIndex(df_calendar['date']).month%12 + 3)//3
df_calendar['season_l'] = pd.cut(df_calendar['season'], 4, labels=["winter", "spring", "summer","autum"])

Replacing null values with null dates

In [38]:
df_calendar_not_nulls = df_calendar.dropna(subset=['adjusted_price', 'price'])

## Understanding the difference between adjusted price and price

In [40]:
# example = [sub(r'[^\d.]', '', r['price']) for i, r in df_calendar_temp.iterrows()]
df_calendar['price'] = [float(sub(r'[^\d.]', '', r['price'])) for i, r in df_calendar.iterrows()]

TypeError: expected string or bytes-like object

In [22]:
df_calendar['adjusted_price'] = [float(sub(r'[^\d.]', '', r['adjusted_price'])) for i, r in df_calendar.iterrows()]

TypeError: expected string or bytes-like object

In [ ]:
df_calendar['difference_price_adjusted'] = df_calendar['price'] - df_calendar['adjusted_price'] 

In [ ]:
df_calendar[df_calendar['difference_price_adjusted']<0]


Conclusion: there is a difference between adjusted price and actual price. We need to further investigate

## Which areas of have the most Airbnb properties, and which are the most expensive?

In [ ]:
df_calendar[df_calendar['difference_price_adjusted']<0]